In [ ]:
import os
import json
from PIL import Image
from pathlib import Path


In [ ]:
def split_image_into_9_parts(image):
    """
    Split a 1536x1536 image into 9 parts (3x3 grid).
    Each part will be 512x512.
    Returns a list of 9 image parts.
    """
    width, height = image.size
    if width != 1536 or height != 1536:
        # Resize if needed
        image = image.resize((1536, 1536), Image.Resampling.LANCZOS)
    
    segment_size = 512
    parts = []
    
    for i in range(3):
        for j in range(3):
            left = j * segment_size
            top = i * segment_size
            right = left + segment_size
            bottom = top + segment_size
            part = image.crop((left, top, right, bottom))
            parts.append(part)
    
    return parts

def combine_4_parts_into_image(parts, indices):
    """
    Combine 4 parts (specified by indices) into a 1024x1024 image.
    Parts are arranged in a 2x2 grid.
    """
    # Create a new 1024x1024 RGB image
    new_image = Image.new('RGB', (1024, 1024))
    
    # Resize each part to 512x512 and place in 2x2 grid
    positions = [(0, 0), (512, 0), (0, 512), (512, 512)]
    
    for idx, pos in zip(indices, positions):
        if idx < len(parts):
            resized_part = parts[idx].resize((512, 512), Image.Resampling.LANCZOS)
            # Ensure RGB mode
            if resized_part.mode != 'RGB':
                resized_part = resized_part.convert('RGB')
            new_image.paste(resized_part, pos)
    
    return new_image

def process_image(image_path, output_folder_path, folder_name, base_filename):
    """
    Process a single image: split into 9 parts, create 4 new images.
    Returns list of output filenames.
    """
    # Load and convert to RGB
    img = Image.open(image_path)
    if img.mode != 'RGB':
        img = img.convert('RGB')
    
    # Split into 9 parts
    parts = split_image_into_9_parts(img)
    
    # Create 4 images from different 2x2 combinations
    # Image 1: parts 0,1,3,4 (top-left 2x2)
    # Image 2: parts 1,2,4,5 (top-right 2x2)
    # Image 3: parts 3,4,6,7 (bottom-left 2x2)
    # Image 4: parts 4,5,7,8 (bottom-right 2x2)
    combinations = [
        [0, 1, 3, 4],  # top-left
        [1, 2, 4, 5],  # top-right
        [3, 4, 6, 7],  # bottom-left
        [4, 5, 7, 8]   # bottom-right
    ]
    
    output_filenames = []
    base_name = Path(base_filename).stem  # Remove extension
    
    for i, combo in enumerate(combinations):
        new_image = combine_4_parts_into_image(parts, combo)
        output_filename = f"{base_name}_combo_{i+1}.jpeg"
        output_path = os.path.join(output_folder_path, output_filename)
        new_image.save(output_path, 'JPEG', quality=95)
        output_filenames.append(output_filename)
    
    return output_filenames



In [ ]:
def process_folders(original_folder, output_folder, folders_to_process, class_mapping):
    """
    Process all specified folders and create preprocessed images with JSON labels.
    """
    # Create output folder
    os.makedirs(output_folder, exist_ok=True)
    
    # Store all labels for JSON file
    all_labels = []
    
    # Process each folder
    for folder_name in folders_to_process:
        print(f"Processing folder: {folder_name}")
        
        # Create output subfolder
        output_subfolder = os.path.join(output_folder, folder_name)
        os.makedirs(output_subfolder, exist_ok=True)
        
        # Get source folder path
        source_folder = os.path.join(original_folder, folder_name)
        
        if not os.path.exists(source_folder):
            print(f"Warning: Source folder {source_folder} does not exist. Skipping.")
            continue
        
        # Get class index
        class_idx = class_mapping.get(folder_name)
        if class_idx is None:
            print(f"Warning: No class mapping for {folder_name}. Skipping.")
            continue
        
        # Get all image files
        image_extensions = {'.jpeg', '.jpg', '.png', '.JPEG', '.JPG', '.PNG'}
        image_files = [f for f in os.listdir(source_folder) 
                      if any(f.endswith(ext) for ext in image_extensions)]
        image_files.sort()
        
        print(f"  Found {len(image_files)} images")
        
        # Process each image
        for img_file in image_files:
            source_path = os.path.join(source_folder, img_file)
            
            try:
                # Process image and get output filenames
                output_filenames = process_image(
                    source_path, 
                    output_subfolder, 
                    folder_name, 
                    img_file
                )
                
                # Add labels for each output image
                for output_filename in output_filenames:
                    relative_path = f"{folder_name}/{output_filename}"
                    all_labels.append([relative_path, class_idx])
                
            except Exception as e:
                print(f"  Error processing {img_file}: {e}")
                continue
        
        print(f"  Completed processing {folder_name}")
    
    # Create JSON file with labels
    json_path = os.path.join(output_folder, 'dataset.json')
    json_data = {"labels": all_labels}
    
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(json_data, f, indent=2, ensure_ascii=False)
    
    print(f"\nProcessing complete!")
    print(f"Output folder: {output_folder}")
    print(f"Total images created: {len(all_labels)}")
    print(f"JSON file saved: {json_path}")
    
    return json_path



In [ ]:
pwd

In [ ]:
original_images_folder = '/home/david/mnt/ssd_2_sata/python/phd/datasets/original/o_bc_left'
output_folder = '/home/david/mnt/ssd_2_sata/python/phd/datasets/preprocessed/imagenet_9to4_1024x1024'

types_dict = {'Ultra_Co11': 'средние зерна',
              'Ultra_Co25': 'мелкие зерна',
              'Ultra_Co8': 'средне-мелкие зерна',
              'Ultra_Co6_2': 'крупные зерна',
              'Ultra_Co15': 'средне-мелкие зерна'}

# Class mapping: folder name to class index
class_mapping = {
    'Ultra_Co11': 1,
    'Ultra_Co25': 0,
    'Ultra_Co6_2': 2
}

folders_to_process = ['Ultra_Co11', 'Ultra_Co25', 'Ultra_Co6_2']

# Run the processing
json_path = process_folders(
    original_images_folder,
    output_folder,
    folders_to_process,
    class_mapping
)

